In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
input_path = os.getenv("input_path")


In [3]:
with open (input_path, 'r', encoding='utf-8') as file:
    text = file.read()

In [4]:
print("Length of the input text: ", len(text))

Length of the input text:  1115393


In [6]:
print("First 500 characters of the input text: \n\n")
print(text[:500])  

First 500 characters of the input text: 


First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor
